# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
data = pd.read_csv('../Cities_Weather_Info.csv')
data

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,port alfred,-33.5906,26.8910,20.10,61.0,20.0,3.13,ZA
1,1,mataura,-46.1927,168.8643,10.82,89.0,100.0,3.72,NZ
2,2,surt,31.2089,16.5887,21.47,61.0,22.0,6.37,LY
3,3,vila velha,-20.3297,-40.2925,24.41,85.0,75.0,1.03,BR
4,6,east london,-33.0153,27.9116,21.52,46.0,0.0,7.72,ZA
...,...,...,...,...,...,...,...,...,...
450,493,muli,2.9167,73.5667,29.52,66.0,91.0,2.43,MV
451,494,tuy hoa,13.0833,109.3000,27.11,84.0,47.0,4.32,VN
452,495,srednekolymsk,67.4500,153.6833,-8.15,95.0,100.0,4.99,RU
453,496,giresun,40.5000,38.5000,11.29,59.0,99.0,4.06,TR


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
#store latitude and longitude as locations
locations = data[["Latitude", "Longitude"]]
humidity = data["Humidity"]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False,max_intensity=max(data["Humidity"]),
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:

ideal_dataframe = data.loc[data['Humidity'] > 30]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Humidity'] < 70]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Max Temp'] < 32]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Max Temp'] > 26]
ideal_dataframe = ideal_dataframe.loc[ideal_dataframe['Cloudiness'] < 30]
ideal_dataframe


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
49,59,quatre cocos,-20.2078,57.7625,27.83,67.0,10.0,6.13,MU
66,77,salalah,17.0151,54.0924,31.05,62.0,20.0,5.66,OM
86,98,hue,16.4667,107.6000,31.06,62.0,20.0,3.09,VN
113,127,tongzi,28.7829,105.0877,26.04,58.0,4.0,2.33,CN
223,245,erzin,36.9559,36.2000,26.87,32.0,0.0,2.50,TR
296,327,miandrivazo,-19.5167,45.4667,31.78,47.0,6.0,2.19,MG
378,419,petauke,-14.2426,31.3253,27.03,46.0,20.0,3.18,ZM
379,420,mogapinyana,-22.3667,27.6000,26.05,47.0,23.0,4.81,BW


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df = ideal_dataframe
hotel_df["Hotel Name"] = ""

params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City Name"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotel_name = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    hotel_name = hotel_name.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"Could not find a hotel in {city}")

# Visualize to confirm lat lng appear
hotel_df

,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
49,59,quatre cocos,-20.2078,57.7625,27.83,67.0,10.0,6.13,MU,The Ravenala Attitude Hotel
66,77,salalah,17.0151,54.0924,31.05,62.0,20.0,5.66,OM,Hilton Salalah Resort
86,98,hue,16.4667,107.6000,31.06,62.0,20.0,3.09,VN,Saigon Morin Hotel
113,127,tongzi,28.7829,105.0877,26.04,58.0,4.0,2.33,CN,Shijichun Hotel
223,245,erzin,36.9559,36.2000,26.87,32.0,0.0,2.50,TR,Büyük Osmaniye Oteli
296,327,miandrivazo,-19.5167,45.4667,31.78,47.0,6.0,2.19,MG,Hôtel Arc En Ciel Miandrivazo
378,419,petauke,-14.2426,31.3253,27.03,46.0,20.0,3.18,ZM,Chimwemwe Lodge
379,420,mogapinyana,-22.3667,27.6000,26.05,47.0,23.0,4.81,BW,MLK B&B


In [44]:
hotel_df.head()
hotel_df = hotel_df.rename(columns = {"City Name": "City"})

In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [46]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…